In [1]:
import warnings
warnings.filterwarnings('ignore')
from db.queries import get_user_forecast_data, row_to_config
from db.utilities import env, logger
from etl.etl import *

from modeling.autoarima import *

In [3]:
logger.info(f"Running on {env}")

INFO:db.utilities:Running on QA


In [7]:
ufm_df = get_user_forecast_data(databrick_task_id = databrick_task_id)
ufm_df.head()

StartDate     EndDate Parameters Region     Status  ForecastMethodID  \
0  2025-03-01  2026-03-31    (1,1,1)     EC  Completed                 1   

   UserForecastMethodID                                       CustomerJSON  \
0                    68  {"CustomerID": ["8477556503","8688980326","760...   

                                             varJSON Method  DatabrickID  
0  {"VariableID": ["OffPeakConsumption","PeakCons...  ARIMA            1

In [9]:
row = ufm_df.iloc[0]
ufm_config = row_to_config(row)
ufm_config

ForecastConfig(forecast_method_id=1, forecast_method_name='ARIMA', model_parameters='(1,1,1)', region='EC', status='Completed', user_forecast_method_id=68, start_date=datetime.date(2025, 3, 1), end_date=datetime.date(2026, 3, 31), databrick_id=1)

In [11]:
from etl.etl import *

In [13]:
metadata = extract_metadata(ufm_df)
customer_ids = parse_json_column(ufm_df, "CustomerJSON")
variable_ids = parse_json_column(ufm_df, "varJSON", key="VariableID")
columns_mapping = generate_combinations()


INFO:root:Generated 255 column combinations.


In [15]:
logging.info(f"Customer IDs: {customer_ids}")
logging.info(f"Variable IDs: {variable_ids}")
logging.info(f"✅ Total column combinations: {len(columns_mapping)}")

INFO:root:Customer IDs: ['9959892919', '8431776530', '7340194598', '5796109807', '8618787245', '9940398550', '5049230573', '6126061820', '9618596053', '6443004614', '7604815667', '8688980326', '7298797010', '7323323155', '9572449312', '8477556503', '9437454176', '8059637149', '7397925811', '8076461989', '6174962014', '5945054457']
INFO:root:Variable IDs: ['OffPeakConsumption', 'PeakConsumption', 'StandardConsumption']
INFO:root:✅ Total column combinations: 255


In [17]:
selected_columns = find_matching_combination(columns_mapping)

INFO:root:Exact match found for: frozenset({'NonTOUConsumption', 'Block3Consumption', 'OffPeakConsumption', 'Block1Consumption', 'StandardConsumption', 'Block2Consumption', 'PeakConsumption', 'Block4Consumption'})


In [19]:
df = load_and_prepare_data(ufmd=ufm_config.user_forecast_method_id, method= ufm_config.forecast_method_name, environment=env)

INFO:root:📂 Loading dataset from data/QA/PredictiveInputDataARIMA.csv
INFO:root:✅ Raw dataset loaded.
INFO:root:🔄 Converted 'CustomerID' to string.
INFO:root:🔢 Data sorted by 'PodID' and 'ReportingMonth'.
INFO:root:✅ Raw dataset cleaned.
INFO:root:🧹 Data cleaned using 'clean_dataframe'.


In [ ]:
from dml.dml import *

In [29]:
customer_ids, pod_ids = get_unique_list_of_customer_and_pod(df)

INFO:root:🧮 Forecasting for 18


In [31]:
# These variables should come from user input / config
StartDate = ufm_config.start_date
EndDate = ufm_config.end_date
Hyper_Parameters = ufm_config.model_parameters

forecast_dates = get_forecast_range(StartDate, EndDate)
arima_order, seasonal_order = extract_sarimax_params(Hyper_Parameters)

INFO:root:📅 Forecast period: 2025-03-01 00:00:00 to 2026-03-01 00:00:00
INFO:root:📌 Parsed ARIMA Order: (1, 1, 1), Seasonal Order: (0, 0, 0)


In [ ]:
if df.empty:
    logging.error("🚫 DataFrame is empty. Check input filters or data source.")
else:
    customer_ids, pod_ids = get_unique_list_of_customer_and_pod(df)

    # These variables should come from user input / config
    StartDate = ufm_config.start_date
    EndDate = ufm_config.end_date
    Hyper_Parameters = ufm_config.model_parameters

    forecast_dates = get_forecast_range(StartDate, EndDate)
    arima_order, seasonal_order = extract_sarimax_params(Hyper_Parameters)

    # Extract actuals range
    latest_actual_date = df.index.max()
    logging.info(f"📍 Last actuals month in data: {latest_actual_date.strftime('%Y-%m')}")

In [ ]:
melted_df = get_melted_df(df)

In [ ]:
from visualization.time_series_tests import *
from visualization.clustering import *

In [ ]:
plot_consumption_trends(melted_df)

📊 Visualization: Customer-Level Electricity Consumption Trends
This chart shows how different types of electricity consumption evolve over time for a sample of top customers:

🧩 Insights:
Seasonal patterns are visible — especially in PeakConsumption and Block1Consumption.

Customer behavior varies — some use more off-peak or flat-rate (NonTOU) power.

Trends help you:

Choose which features to forecast.

Spot outliers or data quality issues.

Guide model segmentation (e.g., cluster by consumption type).

In [ ]:
plot_consumption_over_time(df)

📈 Individual Consumption Trends per Type (Top Customers)
Each plot above shows how a specific consumption type evolves monthly for the top 4 customers. Here's what we can observe:

🧠 Key Observations:
PeakConsumption & Block1Consumption show clear periodic trends for some customers.

Block2–Block4Consumption is flat or zero in many cases, indicating:

Low usage customers not reaching higher billing tiers.

Good candidates to drop or impute if doing dimensionality reduction.

NonTOUConsumption is high and consistent for some — suggesting they are on flat-rate tariffs.

🔍 Suggested Next Steps for EDA (Exploratory Data Analysis)
Here’s a breakdown of what we should do to deepen understanding before modeling:

✅ 1. Seasonality & Stationarity Checks
Autocorrelation Plots (ACF, PACF) to detect periodicity.

Augmented Dickey-Fuller Test for stationarity (needed for ARIMA/SARIMA).

✅ 2. Missing Data / Anomaly Detection
Check for missing months or sudden spikes.

Plot z-scores or IQRs for each column to detect anomalies.

✅ 3. Customer Segmentation
Cluster customers based on consumption profiles (e.g., KMeans, t-SNE).

Helps in building segmented models for better accuracy.

✅ 4. Correlation Heatmap
Explore correlation between consumption types.

Can inform dimensionality reduction (e.g., PCA).

✅ 5. Rolling Mean & Variance Plots
Visualize changes in trend or volatility over time.

Essential for model diagnostics.



In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

In [ ]:
ts, single_customer, cust_df = get_single_time_series_for_single_customer(df)

In [ ]:
acf_and_pacf_plots(ts, single_customer)

In [ ]:

# 2. Augmented Dickey-Fuller Test for Stationarity
adf_result = adfuller(ts.dropna())
adf_output = {
    'Test Statistic': adf_result[0],
    'p-value': adf_result[1],
    'Lags Used': adf_result[2],
    'Number of Observations': adf_result[3],
    'Critical Values': adf_result[4]
}
adf_output

✅ 1. ACF & PACF Results – PeakConsumption for Most Active Customer
ACF (Autocorrelation Function) shows clear periodicity and gradual decay — a sign of seasonality.

PACF (Partial Autocorrelation Function) shows a few significant lags, suggesting possible AR components.

✅ 2. Augmented Dickey-Fuller (ADF) Test for Stationarity
📌 Interpretation:
The ADF test statistic is much lower than the 1% threshold.

The p-value is far below 0.05, which means:

✅ The time series is stationary, and differencing is likely not needed.



In [ ]:
plot_correlation_matrix_of_consumption_types(df)

🔗 Correlation Matrix: Consumption Types
📊 Insights:
High correlation between:

PeakConsumption and StandardConsumption → Similar temporal behavior.

Block1 and StandardConsumption → Users often stay within the first tier during standard hours.

Low or no correlation:

Block3, Block4 show weaker correlations — these may be sparsely used or rare.

NonTOUConsumption has moderate correlation with StandardConsumption, which is expected as both relate to consistent usage patterns.

💡 Implications:
Highly correlated features may be redundant in a predictive model.

Dimensionality reduction (e.g., PCA) could be applied.

You could choose a subset of these features to avoid multicollinearity.

In [ ]:
# 4. Rolling Mean & Variance for PeakConsumption of selected customer
ts = cust_df.set_index('ReportingMonth')['PeakConsumption']
rolling_statistics(ts, single_customer, window_size = 6 )

📉 Rolling Statistics: PeakConsumption for Selected Customer
📌 Interpretation:
Rolling Mean shows smooth seasonality with some upward and downward trends, indicating non-constant behavior over time.

Rolling Std Dev (volatility) varies, with peaks suggesting unusual or variable months.

🧠 Why This Matters:
Helps diagnose trend and heteroskedasticity (changing variance).

If using models like ARIMA or LSTM, rolling stats inform:

Whether differencing or transformations are needed.

Where to apply smoothing or anomaly detection.



In [ ]:
customer_segmentation(df)

In [ ]:
from modeling.autoarima import automated_forecasts_for_all_types

In [ ]:
forecast_combined_df = automated_forecasts_for_all_types(df, selected_columns,forecast_dates)

In [ ]:
# forecast_combined_df